<a href="https://colab.research.google.com/github/eftekhar-hossain/SKBI_Training/blob/main/Session_12_(Tf_idf%2BMNB).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center> <h1> <u> <font color='red'> Training on AI and ML with Python (Intermediate) </font> </u> </h1> </center>

#Session-12: TF-IDF and Count Vectorizer (MNB)


###Objective: 
 1. *Apply Count Vectorizer and Tf-idf.*
 2. *Apply Naive Bayes Algorithm.*

 



In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import sklearn
print(sklearn.__version__)

1.2.2


In [ ]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
np.random.seed(42)

CPU times: user 7.5 s, sys: 1.72 s, total: 9.22 s
Wall time: 17.3 s


#Dataset Fetching

In [ ]:
# Download the data
!wget -O reviews.csv https://www.dropbox.com/s/vyhodtbbumedlv2/restaurant-reviews.csv?dl=0

In [ ]:
# %%time 
data = pd.read_csv("reviews.csv")
data = data.dropna()

In [ ]:
print("Size of the total data==> ", len(data))
class_names= data.Sentiment.unique() ## Get the unique values in a specific column
print(class_names)

Size of the total data==>  1431
['positive' 'negative']


In [ ]:
data['training_label']=data.Sentiment.replace({'positive':1,'negative':0})
data.sample(20)

,Reviews,Sentiment,training_label,cleaned
443,আমি দেখেছি খাদ্য স্বাস্থ্যকর এবং পরিবেশ ও অবস্...,positive,1,আমি দেখেছি খাদ্য স্বাস্থ্যকর এবং পরিবেশ ও অবস্...
274,কিছু খাবার সুস্বাদু ছিল। কিছু খাবার এভারেজ মান...,positive,1,কিছু খাবার সুস্বাদু ছিল কিছু খাবার এভারেজ মান...
761,আমি মনে করি না যে আমাকে আবার যেতে হবে।,negative,0,আমি মনে করি না যে আমাকে আবার যেতে হবে
651,স্বাদ ও মূল্য অনুযায়ী কোনো রেস্টুরেন্টে খাওয়া ...,negative,0,স্বাদ ও মূল্য অনুযায়ী কোনো রেস্টুরেন্টে খাওয়া ...
854,"যদিও অর্ডার এবং খাদ্য পেতে অপেক্ষা ছিল,পরিষেবা...",negative,0,যদিও অর্ডার এবং খাদ্য পেতে অপেক্ষা ছিল পরিষেবা...
1397,"বিস্ময়কর স্থান, মহান সেবা ভালো খাবার Ô £ ø√º√...",negative,0,বিস্ময়কর স্থান মহান সেবা ভালো খাবার
200,করররররররা।,positive,1,করররররররা
490,খুব সুন্দর,positive,1,খুব সুন্দর
789,আমরা মনে করি আমরা একটি ক্ষমা বা প্রশংসাসূচক পা...,negative,0,আমরা মনে করি আমরা একটি ক্ষমা বা প্রশংসাসূচক পা...
122,এককথায় পিজাটা ছিল .... cheeseeeeee ..... এটার ...,positive,1,এককথায় পিজাটা ছিল এটার ...


In [ ]:
%%time
# text cleaning

def cleaning_bangla(x): 
  x = re.sub('[^\u0980-\u09FF]',' ', x)
  # Remove quotes from source text
  x = re.sub("'", '', x)
  # create a set of all special characters
  x = x.strip()
  return x 

data['cleaned'] = data['Reviews'].apply(cleaning_bangla)

CPU times: user 24.9 ms, sys: 0 ns, total: 24.9 ms
Wall time: 27.6 ms


In [ ]:
data.head(15)

##Visualize tf-idf and count vectorizer

In [ ]:
tf_idf = TfidfVectorizer(ngram_range=(1,1), min_df=1, tokenizer=lambda x: x.split(),  use_idf = True)
tf_corpus = tf_idf.fit_transform(data.cleaned[:3])
cv = CountVectorizer(ngram_range=(1,1), min_df=1, tokenizer=lambda x: x.split())
cv_corpus = cv.fit_transform(data.cleaned[:3])

In [ ]:
#Here first parameter is the number of documents and second parameter is the number of features

print(tf_corpus.shape, cv_corpus.shape)

print(cv.vocabulary_)

print(cv.get_feature_names_out())

(3, 31) (3, 31)
{'পিজাটা': 23, 'চমৎকার': 16, 'ছিল': 17, 'আমি': 3, 'এবং': 9, 'আমার': 2, 'বন্ধুদের': 25, 'এটি': 7, 'পছন্দ': 22, 'হয়েছে': 30, 'এরাবিয়ান': 10, 'মাস্টারের': 27, 'অভ্যন্তরীণ': 1, 'সজ্জাটা': 28, 'অনন্য': 0, 'এটির': 8, 'খাদ্য': 14, 'সুস্বাদু': 29, 'একটু': 6, 'মসলাযুক্ত': 26, 'তাদের': 21, 'কর্মীরা': 12, 'খুব': 15, 'বন্ধুত্বপূর্ণ': 24, 'কোরিয়ান': 13, 'জামাইকান': 19, 'উপভোগ': 4, 'করার': 11, 'জন্য': 18, 'একটি': 5, 'জায়গা': 20}
['অনন্য' 'অভ্যন্তরীণ' 'আমার' 'আমি' 'উপভোগ' 'একটি' 'একটু' 'এটি' 'এটির'
 'এবং' 'এরাবিয়ান' 'করার' 'কর্মীরা' 'কোরিয়ান' 'খাদ্য' 'খুব' 'চমৎকার' 'ছিল'
 'জন্য' 'জামাইকান' 'জায়গা' 'তাদের' 'পছন্দ' 'পিজাটা' 'বন্ধুত্বপূর্ণ'
 'বন্ধুদের' 'মসলাযুক্ত' 'মাস্টারের' 'সজ্জাটা' 'সুস্বাদু' 'হয়েছে']


In [ ]:
# Convert sparse matrix to dataframe
cv_corpus = pd.DataFrame.sparse.from_spmatrix(cv_corpus)
# Save mapping on which index refers to which words
col_map = {v:k for k, v in cv.vocabulary_.items()}
# Rename each column using the mapping
for col in cv_corpus.columns:
    cv_corpus.rename(columns={col: col_map[col]}, inplace=True)
cv_corpus

,অনন্য,অভ্যন্তরীণ,আমার,আমি,উপভোগ,একটি,একটু,এটি,এটির,এবং,...,তাদের,পছন্দ,পিজাটা,বন্ধুত্বপূর্ণ,বন্ধুদের,মসলাযুক্ত,মাস্টারের,সজ্জাটা,সুস্বাদু,হয়েছে
0,0,0,1,1,0,0,0,1,0,1,...,0,1,1,0,1,0,0,0,0,1
1,1,1,0,0,0,0,1,0,1,1,...,1,0,0,1,0,1,1,1,1,0
2,0,0,0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
print(tf_idf.vocabulary_) ## Returns the vocabulary with index

print(tf_idf.get_feature_names_out()) ## Return the feature words


{'পিজাটা': 23, 'চমৎকার': 16, 'ছিল': 17, 'আমি': 3, 'এবং': 9, 'আমার': 2, 'বন্ধুদের': 25, 'এটি': 7, 'পছন্দ': 22, 'হয়েছে': 30, 'এরাবিয়ান': 10, 'মাস্টারের': 27, 'অভ্যন্তরীণ': 1, 'সজ্জাটা': 28, 'অনন্য': 0, 'এটির': 8, 'খাদ্য': 14, 'সুস্বাদু': 29, 'একটু': 6, 'মসলাযুক্ত': 26, 'তাদের': 21, 'কর্মীরা': 12, 'খুব': 15, 'বন্ধুত্বপূর্ণ': 24, 'কোরিয়ান': 13, 'জামাইকান': 19, 'উপভোগ': 4, 'করার': 11, 'জন্য': 18, 'একটি': 5, 'জায়গা': 20}
['অনন্য' 'অভ্যন্তরীণ' 'আমার' 'আমি' 'উপভোগ' 'একটি' 'একটু' 'এটি' 'এটির'
 'এবং' 'এরাবিয়ান' 'করার' 'কর্মীরা' 'কোরিয়ান' 'খাদ্য' 'খুব' 'চমৎকার' 'ছিল'
 'জন্য' 'জামাইকান' 'জায়গা' 'তাদের' 'পছন্দ' 'পিজাটা' 'বন্ধুত্বপূর্ণ'
 'বন্ধুদের' 'মসলাযুক্ত' 'মাস্টারের' 'সজ্জাটা' 'সুস্বাদু' 'হয়েছে']


In [ ]:
# Convert sparse matrix to dataframe
tf_corpus = pd.DataFrame.sparse.from_spmatrix(tf_corpus)
# Save mapping on which index refers to which words
col_map = {v:k for k, v in tf_idf.vocabulary_.items()}
# Rename each column using the mapping
for col in tf_corpus.columns:
    tf_corpus.rename(columns={col: col_map[col]}, inplace=True)
tf_corpus

,অনন্য,অভ্যন্তরীণ,আমার,আমি,উপভোগ,একটি,একটু,এটি,এটির,এবং,...,তাদের,পছন্দ,পিজাটা,বন্ধুত্বপূর্ণ,বন্ধুদের,মসলাযুক্ত,মাস্টারের,সজ্জাটা,সুস্বাদু,হয়েছে
0,0.000000,0.000000,0.334689,0.334689,0.000000,0.000000,0.000000,0.334689,0.000000,0.197673,...,0.000000,0.334689,0.334689,0.000000,0.334689,0.000000,0.000000,0.000000,0.000000,0.334689
1,0.263993,0.263993,0.000000,0.000000,0.000000,0.000000,0.263993,0.000000,0.263993,0.155918,...,0.263993,0.000000,0.000000,0.263993,0.000000,0.263993,0.263993,0.263993,0.263993,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.355173,0.355173,0.000000,0.000000,0.000000,0.209771,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


#Model Training

In [ ]:
corpus, labels = data.cleaned, data.training_label

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(corpus,labels,train_size = 0.8, test_size = 0.2,random_state =0)

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline

In [ ]:
%%time
vec = TfidfVectorizer(ngram_range=(1,1), min_df=1, tokenizer=lambda x: x.split(), use_idf=True)
# vec = CountVectorizer(ngram_range=(1,1), min_df=1, tokenizer=lambda x: x.split())
clf = MultinomialNB()
pipe = make_pipeline(vec, clf)
pipe.fit(X_train, y_train)

CPU times: user 41.4 ms, sys: 0 ns, total: 41.4 ms
Wall time: 43.1 ms


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x7f7065ff8310>)),
                ('multinomialnb', MultinomialNB())])

In [ ]:
from sklearn import metrics
def print_report(pipe):
    y_pred = pipe.predict(X_test)
    report = metrics.classification_report(y_test, y_pred,
        target_names=['negative','positive'])
    cm= confusion_matrix(y_test, y_pred)
    print(report)
    print(cm)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

print_report(pipe)

              precision    recall  f1-score   support

    negative       0.81      0.91      0.86       155
    positive       0.88      0.75      0.81       132

    accuracy                           0.84       287
   macro avg       0.84      0.83      0.83       287
weighted avg       0.84      0.84      0.83       287

[[141  14]
 [ 33  99]]
accuracy: 0.836


In [ ]:
def print_prediction(doc):
    y_pred = pipe.predict_proba([doc])[0]
    for target, prob in zip(['negative','positive'], y_pred):
        print("{:.3f} {}".format(prob, target))

doc = data.cleaned[1]
print(doc)
print_prediction(doc)

এরাবিয়ান মাস্টারের  অভ্যন্তরীণ সজ্জাটা অনন্য  এটির খাদ্য সুস্বাদু এবং একটু মসলাযুক্ত    তাদের কর্মীরা খুব বন্ধুত্বপূর্ণ
0.254 negative
0.746 positive


#Model Explanation

In [ ]:
%%time
vec = TfidfVectorizer(ngram_range=(1,1), min_df=1, tokenizer=lambda x: x.split(), use_idf=True)
# vec = CountVectorizer(ngram_range=(1,1), min_df=1, tokenizer=lambda x: x.split())
clf = LogisticRegressionCV()
pipe = make_pipeline(vec, clf)
pipe.fit(X_train, y_train)

CPU times: user 1.69 s, sys: 9.23 ms, total: 1.7 s
Wall time: 2.22 s


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x7f7065ff8790>)),
                ('logisticregressioncv', LogisticRegressionCV())])

In [ ]:
!pip install eli5

In [ ]:
eli5.show_weights(clf, top =(20,20),
                  feature_names=vec.get_feature_names_out(),
                  target_names=class_names)

Weight?,Feature
+5.069,ভাল
+3.639,চমৎকার
+3.598,অসাধারণ
+3.221,সেরা
+2.808,সুস্বাদু
+2.603,সন্ত্রস্ত
+2.460,খাবার
+2.452,বার্গার
+2.214,কোরিয়ান
+2.210,ভালবাসি


In [ ]:
eli5.show_prediction(clf, data.cleaned[0],feature_names=vec.get_feature_names_out(), vec = vec,
                     target_names=['negative','positive'])